In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../input/titanic/train.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())


In [ ]:
df.drop(['PassengerId', 'Name'], axis = 1, inplace = True)#useless infos
df.head()

In [ ]:
df['Age'] = df['Age'].fillna(df.groupby('Title')['Age'].transform('mean'))


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])

In [ ]:
df.head()

In [ ]:
df.drop(['Title'], axis = 1, inplace = True)

In [ ]:
df['Ticket'].value_counts()

In [ ]:
import re
ticket_vals = df['Ticket'].map(lambda ticket: re.split(r'(\d+)', ticket))

In [ ]:
fixed_tickets = []
for ticket in ticket_vals:
    check = False
    for val in ticket:
        if val.isnumeric():
            check = True
    if not check: print(ticket)
    if ticket == ['LINE']:
        fixed_tickets.append(-1)
    else:
        for i in range(len(ticket) - 1, 0, -1):
            if ticket[i].isnumeric():
               # print(ticket[i])
                fixed_tickets.append(ticket[i])
                break
print(len(set(fixed_tickets)), len(ticket_vals))

In [ ]:
df['Ticket'] = np.array(fixed_tickets)
df['Ticket'].value_counts()

In [ ]:
df.head()

In [ ]:
df['Embarked'] = le.fit_transform(df['Embarked'])
df.head()

In [ ]:
df['Cabin'].value_counts()

In [ ]:
df['Cabin'].isnull().sum() / len(df['Cabin']) * 100

77% empty

In [ ]:
df.drop(['Cabin'], axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
X = df.drop(['Survived'], axis = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
sc = StandardScaler()
scaled = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(scaled, df['Survived'], test_size = 0.3, random_state = 0)
print(len(y_train), len(y_test))

In [ ]:
y_train.value_counts()

In [ ]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression()
lr.fit(X_train, y_train)

In [ ]:
preds = lr.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(preds, y_test))

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
              {'penalty':['none', 'l2']},
              {'C':[0.001, 0.01, 0.1, 1, 10, 100]}]

grid_search = GridSearchCV(estimator = lr,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0,
                          error_score = 'raise')


grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression(C = 0.01)
lr.fit(X_train, y_train)

In [ ]:
preds = lr.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(preds, y_test))

In [ ]:
lr = LogisticRegression(C = 0.01)
lr.fit(scaled, df['Survived'])

In [ ]:
test = pd.read_csv('../input/titanic/test.csv')
test.head()

In [ ]:
test['Title'] = test['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
test['Age'] = test['Age'].fillna(test.groupby('Title')['Age'].transform('mean'))


In [ ]:
test.drop(['Name', 'Cabin'], axis = 1, inplace = True)
test.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test['Sex'] = le.fit_transform(test['Sex'])

In [ ]:
import re
ticket_vals = test['Ticket'].map(lambda ticket: re.split(r'(\d+)', ticket))

In [ ]:
fixed_tickets = []
for ticket in ticket_vals:
    check = False
    for val in ticket:
        if val.isnumeric():
            check = True
    if not check: print(ticket)
    if ticket == ['LINE']:
        fixed_tickets.append(-1)
    else:
        for i in range(len(ticket) - 1, 0, -1):
            if ticket[i].isnumeric():
               # print(ticket[i])
                fixed_tickets.append(ticket[i])
                break
print(len(set(fixed_tickets)), len(ticket_vals))

In [ ]:
test['Ticket'] = np.array(fixed_tickets)
test['Ticket'].value_counts()

In [ ]:
test['Embarked'] = le.fit_transform(test['Embarked'])
test.head()

In [ ]:
test.isna().sum()

In [ ]:
test['Age'].mean()

In [ ]:
test['Age'] = test['Age'].fillna(30)

In [ ]:
test['Fare'].mean()

In [ ]:
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

In [ ]:
test.isna().sum()

In [ ]:
test.drop(['Title'], axis = 1, inplace = True)

In [ ]:
test.head()

In [ ]:
X = test.drop(['PassengerId'], axis = 1)
sc = StandardScaler()
scaled_X = sc.fit_transform(X)
scaled_X

In [ ]:
scaled_X.shape

In [ ]:
preds = lr.predict(scaled_X)

In [ ]:
preds.shape

In [ ]:
test["Survived"] = preds
test.head()

In [ ]:
result = test[['PassengerId', 'Survived']]
result.head()

In [ ]:
result.to_csv('submission2.csv', index = False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RF
clf = RF()
clf.fit(scaled, df['Survived'])

In [ ]:
preds = clf.predict(scaled_X)

In [ ]:
test["Survived"] = preds
test.head()

In [ ]:
result = test[['PassengerId', 'Survived']]
result.head()